In [130]:
# sys
import sys

# warning off
import warnings
# IO
import json

# logging
from loguru import logger

# set logger level
logger.remove(0)
logger.add(sys.stderr, level="INFO")

warnings.filterwarnings("ignore")

In [283]:
# reading annotation files
with open('dataset/annotations.json') as fp:
    raw_annotations = json.load(fp)

with open('dataset/annotated-osm-entities.json') as fp:
    second_annotator = json.load(fp)

In [284]:
first_annotator = {}
for annotation_record in raw_annotations:
    page = None
    annotation = []
    for info in annotation_record:
        if 'page' in info.keys():
            page = str(info['page'])
        elif 'type' in info.keys() and 'osm_id' in info.keys():
            annotation.append(f"{info['type']}:{info['osm_id']}")
        else:
            logger.error(f'structural issues: {info}')

    if page is not None:
        first_annotator[page] = annotation

logger.info(f'total annotations: {len(first_annotator)}')

2024-07-14 17:34:44.628 | INFO     | __main__:<module>:16 - total annotations: 2400
2024-07-14 17:34:44.628 | INFO     | __main__:<module>:16 - total annotations: 2400


In [285]:
first_annotator['6']

['way:69366108', 'way:69366081']

In [286]:
second_annotator['6']

['way:69366108', 'way:69366081']

In [287]:
list(set(second_annotator['6']).intersection(first_annotator['6']))

['way:69366081', 'way:69366108']

In [293]:
matched = []
not_matched_from_first = []
not_matched_dict_first = {}
not_matched_from_second = []
not_matched_dict_second = {}

status_list = []

# agreed_first_and_second =[]
first_found = []
second_found = []
total = []
for poi_id, values in second_annotator.items():
    if poi_id not in first_annotator.keys():
        logger.info(f'not annotated by first annotator: {poi_id}')
        continue
    total.append(poi_id)
    first_values = first_annotator[poi_id]
    if len(values) > 0:
        second_found.append(poi_id)
    if len(first_values) > 0:
        first_found.append(poi_id)
    shared_osm_ids = list(set(second_annotator[poi_id]).intersection(first_annotator[poi_id]))
    matched.extend(shared_osm_ids)
    not_matched_from_first.extend([f for f in first_values if f not in shared_osm_ids and 'flag' not in f])
    not_matched_dict_first[poi_id] = [f for f in first_values if f not in shared_osm_ids and 'flag' not in f]
    not_matched_from_second.extend([s for s in values if s not in shared_osm_ids])
    not_matched_dict_second[poi_id] = [s for s in values if s not in shared_osm_ids]
    
    if len(shared_osm_ids) == max(len(second_annotator[poi_id]), len(first_annotator[poi_id])):
        status_list.append('FULLY_MATCHED')
    elif len(shared_osm_ids) > 0:
        status_list.append('PARTIALLY_MATCHED')
    else:
        status_list.append('NOT_MATCHED')

2024-07-14 17:37:42.879 | INFO     | __main__:<module>:15 - not annotated by first annotator: 20
2024-07-14 17:37:42.879 | INFO     | __main__:<module>:15 - not annotated by first annotator: 20


In [289]:
shared_founds = list(set(second_found).intersection(first_found))
logger.info(f'total overlap: {len(total)} - first founds: {len(first_found)} - second founds: {len(second_found)} - shared: {len(shared_founds)}')

2024-07-14 17:34:46.941 | INFO     | __main__:<module>:2 - total overlap: 497 - first founds: 308 - second founds: 291 - shared: 279
2024-07-14 17:34:46.941 | INFO     | __main__:<module>:2 - total overlap: 497 - first founds: 308 - second founds: 291 - shared: 279


In [290]:
logger.info(f'matched: {len(matched)} - not matched from first: {len(not_matched_from_first)} - not matched from second: {len(not_matched_from_second)}')

2024-07-14 17:34:47.432 | INFO     | __main__:<module>:1 - matched: 345 - not matched from first: 69 - not matched from second: 42
2024-07-14 17:34:47.432 | INFO     | __main__:<module>:1 - matched: 345 - not matched from first: 69 - not matched from second: 42


In [291]:
[f for f in first_found if f not in second_found]

['5',
 '37',
 '47',
 '54',
 '79',
 '82',
 '105',
 '111',
 '133',
 '136',
 '154',
 '164',
 '185',
 '200',
 '207',
 '213',
 '219',
 '223',
 '225',
 '230',
 '238',
 '242',
 '250',
 '259',
 '271',
 '308',
 '313',
 '459',
 '460']

In [294]:
from collections import Counter
Counter(status_list)

Counter({'FULLY_MATCHED': 407, 'PARTIALLY_MATCHED': 31, 'NOT_MATCHED': 59})

In [296]:
407/len(total)*100

81.89134808853119

In [297]:
(407+31)/len(total)*100

88.12877263581488